This notebook loads and displays the results of the GEE analyses.

In [1]:
# Add the parent directory of the current working directory to the Python path at runtime. 
# In order to import modules from the src directory.
import os
import sys 

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [2]:
import glob
import pickle
from prettytable import PrettyTable
from src.stat_utils import *

results_dir = os.path.join(parent_dir, 'results/statistics')
first_model_file = os.path.join(results_dir, 'gee_first.pkl')
full_model_file = os.path.join(results_dir, 'gee_full.pkl')

In [3]:
def add_row_to_table(session, results, table):
    predictors = results.model.exog_names
    for i, predictor in enumerate(predictors):
        coefficient = results.params[predictor]
        confidence_interval = [results.conf_int()[0][predictor], results.conf_int()[1][predictor]]
        standard_error = results.standard_errors()[i]
        wald = results.wald_test(predictor, scalar=True)
        chi = wald.statistic
        pvalue = wald.pvalue
        table.add_row([session, predictor, coefficient, confidence_interval, standard_error, chi, pvalue])

    return table

### Results of first session model

In [4]:
with open(first_model_file, 'rb') as f:
    gee_first = pickle.load(f)

print(gee_first.summary())

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                 6000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                 750
                      Estimating Equations   Max. cluster size:                 750
Family:                           Binomial   Mean cluster size:               750.0
Dependence structure:         Exchangeable   Num. iterations:                     6
Date:                     Wed, 17 Jul 2024   Scale:                           1.000
Covariance type:                    robust   Time:                         14:37:22
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                         

In [5]:
print_wald_chi_square(gee_first)

Wald Chi-Square:
+--------------------------------------+--------------------+------------------------+
|               Variable               |     Chi-Square     |        p-value         |
+--------------------------------------+--------------------+------------------------+
|              Intercept               | 21.201510214322358 | 4.134386106069118e-06  |
|        ContrastHeterogeneity         | 18.361082803614064 | 1.8275282694086964e-05 |
|            GridCoarseness            | 17.777698465296414 | 2.4827641305959744e-05 |
| ContrastHeterogeneity:GridCoarseness | 15.26688399705535  | 9.333864182752107e-05  |
+--------------------------------------+--------------------+------------------------+


### Results of full model

In [6]:
with open(full_model_file, 'rb') as f:
    gee_full = pickle.load(f)

print(gee_full.summary())

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                48000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                6000
                      Estimating Equations   Max. cluster size:                6000
Family:                           Binomial   Mean cluster size:              6000.0
Dependence structure:         Exchangeable   Num. iterations:                    15
Date:                     Wed, 17 Jul 2024   Scale:                           1.000
Covariance type:                    robust   Time:                         14:37:22
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                         

In [7]:
print_wald_chi_square(gee_full)

Wald Chi-Square:
+--------------------------------------+--------------------+------------------------+
|               Variable               |     Chi-Square     |        p-value         |
+--------------------------------------+--------------------+------------------------+
|              Intercept               | 117.69888757712704 | 2.018136699568119e-27  |
|        ContrastHeterogeneity         | 150.17554736209928 | 1.5870455850608488e-34 |
|            GridCoarseness            | 127.6901266521975  | 1.3120962546709636e-29 |
|              SessionID               | 29.911719884031275 | 4.521713783939351e-08  |
|   SessionID:ContrastHeterogeneity    | 71.56806961754121  | 2.6785634689184098e-17 |
|       SessionID:GridCoarseness       | 1.4108291925000898 |   0.2349187913464244   |
| ContrastHeterogeneity:GridCoarseness | 169.2756434277779  | 1.0650625639784129e-38 |
+--------------------------------------+--------------------+------------------------+


### Simple effects for significant interactions

In [9]:
table = PrettyTable()
table.field_names = ['session', 'predictor', 'beta-coefficient', '95% confidence interval', 'sdt error', 'Wald chi-square', 'P-value']

files = glob.glob(os.path.join(results_dir, 'gee_*.pkl'))
files.remove(full_model_file)
files.remove(first_model_file)
files = sorted(files)

for i, file in enumerate(sorted(files)):
    with open(file, 'rb') as f:
        results = pickle.load(f)
    table = add_row_to_table(i + 1, results, table)


print(table)

+---------+-----------------------+---------------------+--------------------------------------------+---------------------+--------------------+------------------------+
| session |       predictor       |   beta-coefficient  |          95% confidence interval           |      sdt error      |  Wald chi-square   |        P-value         |
+---------+-----------------------+---------------------+--------------------------------------------+---------------------+--------------------+------------------------+
|    1    |       Intercept       |  1.374067759974428  |  [0.8504068762125688, 1.8977286437362872]  | 0.26717882976035756 | 26.44918004561104  | 2.705633034243971e-07  |
|    1    | ContrastHeterogeneity | -1.4584335093863061 | [-2.019373649151089, -0.8974933696215229]  | 0.28619920783718855 | 25.96787510849494  | 3.471461615896075e-07  |
|    2    |       Intercept       |  1.9753286295858794 |  [1.3820671746272157, 2.5685900845445433]  | 0.30268997779460965 |  42.5875477503321  |